In [1]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import anndata as ad
import muon as mu
import scanpy as sc

import cellij

import torch

if torch.cuda.is_available():
    torch.set_default_tensor_type("torch.cuda.FloatTensor")

# random seed to use throughout the notebook
seed = 123


/Users/martin/miniconda3/envs/cellij/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
mdata = cellij._data.Importer().load_CLL()

In [4]:
mutations = mdata["mutations"].X
mutations = np.nan_to_num(mutations, 0)
mutations = ad.AnnData(
    X=mutations, obs=mdata["mutations"].obs, var=mdata["mutations"].var
)


In [5]:
methylation = mdata["methylation"].X
methylation = np.nan_to_num(methylation, 0)
methylation = ad.AnnData(
    X=methylation, obs=mdata["methylation"].obs, var=mdata["methylation"].var
)


In [6]:
drugs = mdata["drugs"].X
drugs = np.nan_to_num(drugs, np.nanmean(drugs))
drugs = np.clip(drugs, 0, 1)
drugs = ad.AnnData(X=drugs, obs=mdata["drugs"].obs, var=mdata["drugs"].var)


In [7]:
mrna = mdata["mrna"]
sc.pp.highly_variable_genes(
    mrna,
    min_mean=0.01,
    max_mean=8,
    min_disp=1,
    n_top_genes=2000,
    flavor="cell_ranger",
    n_bins=20,
)
mrna = mrna[:, mrna.var.highly_variable]
mrna = ad.AnnData(
    X=np.nan_to_num(mrna.X, np.nanmean(mrna.X)), obs=mrna.obs, var=mrna.var
)


In [8]:
mdata = mu.MuData(
    {"mrna": mrna, "mutations": mutations, "drugs": drugs, "methylation": methylation}
)
mdata


MuData object with n_obs × n_vars = 200 × 6627
  4 modalities
    mrna:	136 x 2000
      var:	'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    mutations:	200 x 69
    drugs:	200 x 310
    methylation:	200 x 4248

In [9]:
model1 = cellij.core.models.MOFA(n_factors=40, sparsity_prior=None)
model1.add_data(data=mdata, na_strategy="knn_by_obs")

model1.fit(
    likelihoods={
        "mrna": "Normal",
        "mutations": "Bernoulli",
        "drugs": "Normal",
        "methylation": "Normal",
    },
    epochs=50,
    verbose_epochs=10,
    learning_rate=0.01,
)


Epoch      0:      370206.64 	 | 
Epoch     10:      335939.48 	 |    9.26%
Epoch     20:      268389.52 	 |   20.11%
Epoch     30:      198950.69 	 |   25.87%
Epoch     40:      150553.72 	 |   24.33%
Epoch     50:      126494.70 	 |   15.98%
Training finished.
